In [1]:
import os
import json
from dotenv import load_dotenv
from langchain.chat_models import init_chat_model
load_dotenv()

True

In [45]:
from langchain_ollama.chat_models import ChatOllama
from langchain_ollama import ChatOllama

model = ChatOllama(
    model="qwen3:8b",
    temperature=0,
    reasoning = False,
)

In [ ]:
few_shot_prompt = """ Complete the following task, make sure to return a python list, no comments, follow the examples:
Q: What are useful visual features for distinguishing a lemur in a photo?
A: There are several useful visual features to tell there is a lemur in a photo:

[
    "four-limbed primate",
    "black, grey, white, brown, or red-brown",
    "wet and hairless nose with curved nostrils",
    "long tail",
    "large eyes",
    "furry bodies",
    "clawed hands and feet"
]

Q: What are useful visual features for distinguishing a television in a photo?
A: There are several useful visual features to tell there is a television in a photo:

[
    "electronic device",
    "black or grey",
    "a large, rectangular screen",
    "a stand or mount to support the screen",
    "one or more speakers",
    "a power cord",
    "input ports for connecting to other devices",
    "a remote control"
]


Q: What are useful features for distinguishing a {category_name} in a photo?
A: There are several useful visual features to tell there is a {category_name} in a photo:
-
"""

In [ ]:
# trial with a single category
desc = model.invoke("""Complete the following task, make sure to return a python list, no comments, follow the examples:
Q: What are useful visual features for distinguishing a lemur in a photo?
A: There are several useful visual features to tell there is a lemur in a photo:

[
    "four-limbed primate",
    "black, grey, white, brown, or red-brown",
    "wet and hairless nose with curved nostrils",
    "long tail",
    "large eyes",
    "furry bodies",
    "clawed hands and feet"
]

Q: What are useful visual features for distinguishing a television in a photo?
A: There are several useful visual features to tell there is a television in a photo:

[
    "electronic device",
    "black or grey",
    "a large, rectangular screen",
    "a stand or mount to support the screen",
    "one or more speakers",
    "a power cord",
    "input ports for connecting to other devices",
    "a remote control"
]


Q: What are useful features for distinguishing a Goldfish in a photo?
A: There are several useful visual features to tell there is a Goldfish in a photo:
""")

In [30]:
text= desc.content.strip().replace('\n', ' ')

start_index = text.find('[') + 1  
end_index = text.find(']')  
content = text[start_index:end_index].strip()
content_list = [item.strip().strip('"') for item in content.split('",')]
content_list



['small, rounded body',
 'scales that reflect light',
 'fins on the sides and tail',
 'gills or an air-breathing system',
 'cold-blooded',
 'usually yellow or orange with black spots']

In [42]:
with open("imagenet_classes.txt", "r") as file:
    text = file.read()

text = text.splitlines()

print("Number of classes:", len(text))
for i in range(len(text)):
    category = text[i].split(" ", 1)[1].strip().split(",")[0]
    print(category)

Number of classes: 1000
tench
goldfish
great white shark
tiger shark
hammerhead
electric ray
stingray
cock
hen
ostrich
brambling
goldfinch
house finch
junco
indigo bunting
robin
bulbul
jay
magpie
chickadee
water ouzel
kite
bald eagle
vulture
great grey owl
European fire salamander
common newt
eft
spotted salamander
axolotl
bullfrog
tree frog
tailed frog
loggerhead
leatherback turtle
mud turtle
terrapin
box turtle
banded gecko
common iguana
American chameleon
whiptail
agama
frilled lizard
alligator lizard
Gila monster
green lizard
African chameleon
Komodo dragon
African crocodile
American alligator
triceratops
thunder snake
ringneck snake
hognose snake
green snake
king snake
garter snake
water snake
vine snake
night snake
boa constrictor
rock python
Indian cobra
green mamba
sea snake
horned viper
diamondback
sidewinder
trilobite
harvestman
scorpion
black and gold garden spider
barn spider
garden spider
black widow
tarantula
wolf spider
tick
centipede
black grouse
ptarmigan
ruffed grouse

In [43]:
text

['n01440764 tench, Tinca tinca',
 'n01443537 goldfish, Carassius auratus',
 'n01484850 great white shark, white shark, man-eater, man-eating shark, Carcharodon carcharias',
 'n01491361 tiger shark, Galeocerdo cuvieri',
 'n01494475 hammerhead, hammerhead shark',
 'n01496331 electric ray, crampfish, numbfish, torpedo',
 'n01498041 stingray',
 'n01514668 cock',
 'n01514859 hen',
 'n01518878 ostrich, Struthio camelus',
 'n01530575 brambling, Fringilla montifringilla',
 'n01531178 goldfinch, Carduelis carduelis',
 'n01532829 house finch, linnet, Carpodacus mexicanus',
 'n01534433 junco, snowbird',
 'n01537544 indigo bunting, indigo finch, indigo bird, Passerina cyanea',
 'n01558993 robin, American robin, Turdus migratorius',
 'n01560419 bulbul',
 'n01580077 jay',
 'n01582220 magpie',
 'n01592084 chickadee',
 'n01601694 water ouzel, dipper',
 'n01608432 kite',
 'n01614925 bald eagle, American eagle, Haliaeetus leucocephalus',
 'n01616318 vulture',
 'n01622779 great grey owl, great gray owl, 

In [ ]:
responses={} 
non_list_format = []
# import json
for i in range(len(text)):
    category = text[i].split(" ", 1)[1].strip().split(",")[0]
    prompt_complete = few_shot_prompt.format(category_name=category)
    response = model.invoke(prompt_complete)
    print(f"Category: {category}")
    print(f"Response: {response}")
    response_text= response.content.strip().replace('\n', ' ')

    start_index = response_text.find('[') + 1  
    end_index = response_text.find(']')  
    content = response_text[start_index:end_index].strip()
    content_list = [item.strip().strip('"') for item in content.split('",')]


    try:
        if type(content_list) is list:
            responses[category] = content_list
    except Exception as r:
        print(f"Error decoding list response for category: {category}, Error: {r}")
        non_list_format.append(category)
        new_prompt = "fix the response to be a valid python list format, heres the response and error look out for ommission on apostrophe for the python strings :" + f"\n{content_list}\nError: {r} \n\n No other explanations, only provide the correct response"
        response2= model.invoke(new_prompt)
        response_text= response2.content.strip().replace('\n', ' ')

        start_index = response_text.find('[') + 1  
        end_index = response_text.find(']')  
        content = response_text[start_index:end_index].strip()
        content_list2 = [item.strip().strip('"') for item in content.split('",')]

        if type(content_list2) is list:
            responses[category] = content_list2

        else:
            non_list_format.append(category)
            print(f"Response Not in list format for category: {category}")

Category: tench
Response: content='[\n    "fish with a streamlined body",\n    "slender and elongated shape",\n    "silver or metallic sheen",\n    "two dorsal fins",\n    "large eyes adapted for low light",\n    "no adipose fin",\n    "fins that are often pointed or rounded",\n    "found in freshwater environments"\n]' additional_kwargs={} response_metadata={'model': 'qwen3:8b', 'created_at': '2025-07-31T13:00:51.76089Z', 'done': True, 'done_reason': 'stop', 'total_duration': 54702341079, 'load_duration': 128435338, 'prompt_eval_count': 245, 'prompt_eval_duration': 31958774501, 'eval_count': 68, 'eval_duration': 22610800670, 'model_name': 'qwen3:8b'} id='run--f83d5abb-c509-4e92-bbeb-c673cb2b28d6-0' usage_metadata={'input_tokens': 245, 'output_tokens': 68, 'total_tokens': 313}
Category: goldfish
Response: content='A: There are several useful visual features to tell there is a goldfish in a photo:\n\n[\n    "small, round body",\n    "fins along the sides and tail",\n    "gills on the si

In [ ]:
with open("descriptor_generation.ipynb", "w") as f:
    json.dump(responses, f, indent=4)